In [3]:
#########################################
#Прототип решения задачи от skillfactory#
#p.s. Я использую xgboost в решении     #
#Его нужно установить заранее           #
#########################################
#xgboost по какой то причине не работает если не сохранять очищенные данные в файл
#Если вам не нужны очищенные данные после работы кода - удалите его
#В конце кода - сохранение результата
#Закоментируйте код в самом конце, если вам не нужен результат

# Время работы кода - 4m 40s при n_jobs = 1

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost
import catboost
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import ast
data = pd.read_csv('data.csv')
data = data.drop(['MlsId', 'mls-id', 'private pool', 'propertyType', 'status', 'zipcode'], axis = 1)

top = ['WA', 'NH', 'MN', 'UT', 'VT', 'MD', 'VA', 'MA', 'NE', 'CO', 'WI', 'NJ', 'FL', 'IA', 'ND', 'ID']
mid = ['GA', 'NC', 'CA', 'SD', 'CT', 'KS', 'DE', 'HI', 'NY', 'RI', 'OR', 'MO', 'MT', 'TN', 'WY', 'ME']

def func_rebuilt(row):
    if row['rebuilt'] == 0:
        return row['built']
    if row['rebuilt'] == '':
        return row['built']
    return row['rebuilt']

def func_1(x):
    if x == '':
        return 0
    return 1

def state(x):
    if x in top:
        return 2
    if x in mid:
        return 1
    return 0

def func_3(x):
    if type(x) != int:
        x = x.replace(x, '1')
    return int(x)

def data_func1(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][0].get('factValue')

def data_func2(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][1].get('factValue')

def data_func3(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][2].get('factValue')

def data_func4(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][3].get('factValue')

def data_func5(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][4].get('factValue')

def data_func6(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][5].get('factValue')

def data_func7(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][6].get('factValue')

def data_func8(x):
    m = []
    a = ast.literal_eval(x)
    a = dict(a[0])
    k = a['data'].get('Distance')
    for i in k:
        h = re.findall(r'\d+',i)
        if len(h) == 0:
            continue
        if len(h) == 1:
            m.append(float(h[0]))
            continue
        d = h[0] + '.' + h[1]
        m.append(float(d))
    if len(m) == 0:
        return 0
    else:
        return min(m)
    
def func_4(x):
    if type(x) != int:
        x = re.findall(r'\d+',x)
        if len(x) > 2:
            x = x[0] + x[1] + x[2]
        elif len(x) > 1:
            x = x[0] + x[1]
        elif len(x) == 1:
            x = x[0]
        if not x:
            return np.nan
    return int(x)

def func_2(x):
    if type(x) != int:
        x = re.findall(r'\d+',x)
        if (len(x) > 1):
            x = x[0]
        elif len(x) == 1:
            x = x[0]
        if not x:
            x = 1
    return int(x)

data['PrivatePool'] = data['PrivatePool'].fillna(0).apply(func_3)
data['fireplace'] = data['fireplace'].fillna(0).apply(func_3)
data['stories'] = data['stories'].fillna(1).apply(func_2)
data['stories'] = pd.to_numeric(data['stories'])
data['baths'] = data['baths'].fillna(1).apply(func_2)
data['beds'] = data['beds'].fillna(1).apply(func_2)
data = data.drop(data.loc[data['city'].isnull()].index)
data = data.drop(data.loc[data['street'].isnull()].index)
data = data.drop(data.loc[data['target'].isnull()].index)
data = data.drop(data.loc[data['sqft'].isnull()].index)
data['sqft'] = data['sqft'].apply(func_4)
data['target'] = data['target'].apply(func_4)
data['built'] = data['homeFacts'].apply(data_func1)
data['rebuilt'] = data['homeFacts'].apply(data_func2)
data['heating'] = data['homeFacts'].apply(data_func3)
data['cooling'] = data['homeFacts'].apply(data_func4)
data['parking'] = data['homeFacts'].apply(data_func5)
data['lotsize'] = data['homeFacts'].apply(data_func6)
data['price'] = data['homeFacts'].apply(data_func7) 
data['distance'] = data['schools'].apply(data_func8)
data = data.drop(['homeFacts', 'schools', 'city', 'street'], axis = 1)
data['cooling'] = data['cooling'].apply(func_1)
data['heating'] = data['heating'].apply(func_1)
data['parking'] = data['parking'].apply(func_1)
data = data.drop(data.loc[data['built'].isnull()].index)
data['built'] = data['built'].apply(func_4)
data = data.drop(data.loc[data['built'].isnull()].index)
data['rebuilt'] = data['rebuilt'].fillna(0)      
data['rebuilt'] = data.apply(lambda row: func_rebuilt(row), axis=1)
data = data.drop(data.loc[data['lotsize'].isnull()].index)
data = data.drop(data.loc[data['price'].isnull()].index)
data['lotsize'] = data['lotsize'].apply(func_4)
data['price'] = data['price'].apply(func_4)
data = data.drop(data.loc[data['lotsize'].isnull()].index)
data = data.drop(data.loc[data['price'].isnull()].index)
data['state'] = data['state'].apply(state)

data = data[data['sqft'] < 500000]
data = data[data['beds'] < 10]
data = data[data['baths'] < 10]
data = data[data['stories'] < 15]

###################################
data.to_csv('model.csv', index=False)
data = pd.read_csv('model.csv')
###################################

X = data.drop('target', axis = 1)
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

rf = RandomForestRegressor(random_state=21, max_depth = 12, min_samples_leaf = 3, 
                           min_samples_split = 2, n_estimators = 50)

xg = xgboost.XGBRegressor(max_depth = 3, min_samples_leaf = 1, 
                          min_samples_split = 1, n_estimators = 40, random_state = 21)

gb = GradientBoostingRegressor(max_depth = 4, min_samples_leaf = 1, 
                               min_samples_split = 2, n_estimators = 75, random_state = 21)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)
xg.fit(X_train, y_train)
xg_pred = xg.predict(X_test)
y_pred = 0.2*rf_pred + 0.6*gb_pred + 0.2*xg_pred

################################################
submission = pd.DataFrame({"target": y_pred})
submission.to_csv("submission.csv", index=False)
################################################